In [0]:
%pip install -e ..

In [0]:
%pip install git+https://github.com/end-to-end-mlops-databricks-3/marvelous

In [0]:
%restart_python

In [0]:
import sys
from pathlib import Path

sys.path.append(str(Path.cwd().parent / "src"))

In [0]:
import mlflow
from loguru import logger
from pyspark.sql import SparkSession

from hotel_reservations.config import ProjectConfig, Tags
from hotel_reservations.models.feature_lookup_model import FeatureLookUpModel

In [0]:
spark = SparkSession.builder.getOrCreate()
tags_dict = {"git_sha": "abcd12345", "branch": "week3"}
tags = Tags(**tags_dict)

In [0]:
config = ProjectConfig.from_yaml(config_path = "../project_config.yml")

In [0]:
# Initialize model
fe_model = FeatureLookUpModel(config=config, tags=tags, spark=spark)

In [0]:
# Create feature table
fe_model.create_feature_table()

In [0]:
# Define last minute booking feature function
fe_model.define_feature_function()

In [0]:
# Load data
fe_model.load_data()

In [0]:
# Perform feature engineering
fe_model.feature_engineering()

In [0]:
# Train the model
fe_model.train()

In [0]:
# Train the model
fe_model.register_model()

In [0]:
# Lets run prediction on the last production model
# Load test set from Delta table

test_set = spark.table(f"{config.catalog_name}.{config.schema_name}.test_set").limit(10)

# Drop feature lookup columns and target
X_test = test_set.drop("avg_price_per_room", "room_type_reserved", "repeated_guest", config.target)

In [0]:
# Make predictions
predictions = fe_model.load_latest_model_and_predict(X_test)

In [0]:
# Display predictions
logger.info(predictions)